## Train Test Split

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
import pandas as pd
from pandas import DataFrame, Series, Timestamp, Timedelta, DatetimeIndex
from pandas.tseries.offsets import DateOffset

pd.set_option("display.max_rows", 5)

In [ ]:
s = DataFrame({"abc": [(0, 1), (1,), (2, 3), (0, 1), (0, 1)]})
s

In [ ]:
pd.api.types.is_categorical_dtype(s)

In [ ]:
pd.get_dummies(s)

In [ ]:
set(s.squeeze().astype(pd.CategoricalDtype()).dtype.categories)

In [ ]:
s = Series([(0, 1), (1,), (2, 3), (0, 1), (0, 1)], name="abc")
s.astype("string").dtype == pd.StringDtype
s

In [ ]:
from tsdm.datasets import Electricity

X = Electricity().dataset
X

In [ ]:
from tsdm.encoders.functional import make_dense_triplets

df = make_dense_triplets(X)
df

In [ ]:
from tsdm.utils import time2int

df["time"] = time2int(df["time"])

In [ ]:
X

In [ ]:
X.columns

In [ ]:
df = X.reset_index()
df["time"] = df["time"].astype(int)
df

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

ds = TimeSeriesDataSet(
    df,
    time_idx="time",
    target=X.columns[:5],
    group_ids=X.columns,
    min_encoder_length=11,
    max_encoder_length=11,
    min_prediction_length=7,
    max_prediction_length=7,
    time_varying_known_reals=X.columns,
)

In [ ]:
dataloader = ds.to_dataloader(batch_size=32)
x, y = next(iter(dataloader))
for key, value in x.items():
    print(f"{key:16s} = {value.size()}")

In [ ]:
from tsdm.utils import make_sparse_triplets

df = make_sparse_triplets(X).reset_index()
df

In [ ]:
from tsdm.utils import time2int

df["time"] = time2int(df["time"])
df

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

ds = TimeSeriesDataSet(
    df,
    time_idx="time",
    target="value",
    group_ids=["variable"],
    min_encoder_length=11,
    max_encoder_length=11,
    min_prediction_length=7,
    max_prediction_length=7,
    time_varying_unknown_reals=["value"],
)

In [ ]:
dataloader = ds.to_dataloader(batch_size=32)
x, y = next(iter(dataloader))
for key, value in x.items():
    print(f"{key:16s} = {value.size()}")